# Finance 5350 

__Computational Financial Modeling__

## Binomial Option Pricing Model

<br>

We can implement the Binomial Option Pricing Model in Python as follows. Let's start with specifing payoff functions for calls and puts. 

<br>

Recall that the payoff for call option is:

$$
\mbox{Call Payoff} = \max{\{S_{t} - K, 0.0 \}}
$$

The put payoff is:

$$
\mbox{Put Payoff} = \max{\{K - S_{t}, 0.0\}}
$$

<br>

We can implement these in code as follows.

In [1]:
## First import numpy
import numpy as np

## Call Payoff Function
def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

## Put Payoff Function
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)


Let's check these with $S = \$100$ and $K_{1} = \$95$ and $K_{2} = \$105$

In [2]:
spot = 100.0
strikes = np.array([95.0, 105.0])
strikes

array([ 95., 105.])

In [3]:
callPO = callPayoff(spot, strikes)
callPO

array([5., 0.])

In [4]:
putPO = putPayoff(spot, strikes)
putPO

array([0., 5.])

They seem to be working just fine!

<br>

- $r = 8\%$
- $\sigma = 30\%$
- $T = 1$ year
- $\delta = 0.0$ (no dividends)
- $n = 1$ (a single period)
- $h = T/n = 1$

Also recall that the formula for the parameters $u$ and $d$ are given by:

$$
\begin{align}
u &= e^{\{(r - \delta)h + \sigma \sqrt{h}\}} \\
d &= e^{\{(r - \delta)h - \sigma \sqrt{h}\}} 
\end{align}
$$

<br>

These can be implemented in Python as follows:

In [5]:
r = 0.08       ## time step 
v = 0.30       ## volatility
q = 0.0        ## dividend
expiry = 1.0
n = 1.0        ## periods
h = expiry / n 
S = 100.0      ## spot price
K = 95.0       ## strike price

u = np.exp((r - q) * h + v * np.sqrt(h))
d = np.exp((r - q) * h - v * np.sqrt(h))

In [6]:
print((u, d))

(1.4622845894342245, 0.8025187979624785)


### Implementing the Single Period Model

Now we can implement the single period model in two ways. Recall that we could write the model in these two forms:

__No-Arbitrage Form:__

$$
f_{0} = \Delta S + B
$$

with 

$$
\Delta = \frac{f_{u} - f_{d}}{S(u - d)}
$$

and

$$
B = e^{-r h}\left[\frac{u f_{d} - d f_{u}}{(u - d)}\right]
$$

Where $f$ stands for either a call or put. Let's see this in action for a call option.

In [7]:
fu = callPayoff(u * S, K)
fd = callPayoff(d * S, K)
D = (fu - fd)/(S * (u - d))                        ## Delta
B = np.exp(-r * h) * ((u * fd - d * fu)/ (u - d))  ## B 
f = S * D + B
f

20.124540120626122

__Risk-Neutral Form__

We saw that we could also put the single period model in risk-neutral form. There we only needed to define one additional parameter $p^{\ast}$, which we do as follows:

$$
p^{\ast} = \frac{e^{(r - \delta)h} - d}{u - d}
$$

<br>

Then the single period model can also be written as:

$$
f_{0} = e^{-rh} \left[(p^{\ast}) f_{u} - (1 - p^{\ast}) f_{d} \right]
$$

<br>

Which we implement in Python as follows:

In [12]:
pstar = (np.exp((r - q) * h) - d) / (u - d)
pstar

0.4255574831883412

In [13]:
f = np.exp(-r * h) * (pstar * fu + (1 - pstar) * fd)
f

20.124540120626122

From this we can see that they are the exact same solution! Yeet!!!

<br>

But we're now experienced Python programmers. Instead of inline code, let's see if we can abstract this away into a function.

In [10]:
def singlePeriodBinomialModel(S, K, r, v, q, T, n, payoff):
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    fu = payoff(u * S, K)
    fd = payoff(d * S, K)
    f = np.exp(-r * h) * (pstar * fu + (1 - pstar) * fd)
    
    return f
    

In [11]:
callPrc = singlePeriodBinomialModel(S, K, r, v, q, expiry, n, callPayoff)
callPrc

20.124540120626122

Yeet!!!

<br>

Also, note that we can now also price the put for free! This is actually an example of polymorphism (which we'll return to). It is possible because functions are first-class objects in Python so we can pass functions to functions. 

In [14]:
putPrc = singlePeriodBinomialModel(S, K, r, v, q, expiry, n, putPayoff)
putPrc

7.82059302735651

### The Multi-Period Binomial Model

Well, great but the single period model is pretty simplistic. We don't think it is sufficient for pricing actual real-world options. It's just a toy model. Recall that we used backward induction to solve recursively for the option price in a multi-period model applying at each set of nodes the single period model. So we can build on the single period model to solve for the option price in a more complex model. How can we implement this in Python code? Let's start by recalling our simplification for European options. 

<br>

Let's start with a two-period model ($n = 2$). The terminal prices on the last nodes of the tree are given as follows:

- $uuS$
- $udS = duS$
- $ddS$

Using the way of thinking embedded in the risk-neutral form of the single period model we can write the option premium as:

$$
f_{0} = e^{-r T} \left[p_{uu} f_{uu} + p_{ud} f_{ud} + p_{dd} f_{dd} \right]
$$

Assuming we could know the risk-neutral probabilities $p_{uu}, p_{ud}, p_{dd}$. 

<br>

Let's start by seeing how to get the terminal prices on the last set up nodes (to which we can apply a payoff function). We can achieve this pretty simply with the following and using a NumPy array:

In [15]:
S = 100.0
K = 95.0
r = 0.08
v = 0.30
q = 0.0
T = 1
n = 2
h = T / n
u = np.exp((r - q) * h + v * np.sqrt(h))
d = np.exp((r - q) * h - v * np.sqrt(h))
pstar = (np.exp((r - q) * h) - d) / (u - d) 

In [17]:
nodes = n + 1
spotT = np.zeros((nodes, ))
spotT

array([0., 0., 0.])

Now we've got space to store the values. Let's fill the slots in with the correct values for the terminal spot prices at the nodes with a simpel for loop construct (starting at the bottom node and working up):

In [18]:
for i in range(nodes): 
    spotT[i] = S * (u ** (i)) * (d ** (n - i))
spotT

array([ 70.87417468, 108.32870677, 165.57665416])

We can spot check these:

In [19]:
d * d * S

70.87417468160731

In [20]:
u * d * S

108.32870676749585

In [21]:
u * u * S

165.5766541569831

Yeet. Now all we need are the probabilities to go with the terminal prices.

#### The Binomial Probability Mass Function

The Binomial Distribution Function is given as follows:

$$
p(i) = \binom{n}{i} \theta^{i} (1 - \theta)^{n - i}
$$

where 

$$
\binom{n}{i} = \frac{n!}{i! (n - i)!}
$$

and 

- $\theta$ is the probability of _success_ (an up move on the tree)
- $i$ is the number of successes in $n$ periods ($i$ is also the index variable in our for loop)

<br>

So if we set $\theta = p^{\ast} = \frac{e^{(r - \delta)h} - d}{u - d}$, then we can treat the binomial distribution as our risk-neutral distribution. We can then get the corresponding risk-neutral probabilities for the terminal nodes as follows (starting at the top of the tree and working down):

- $p(i = 2) = (1) (p^{\ast})^{2} (1 - p^{\ast})^{0} = (p^{\ast})^{2}$
- $p(i = 1) = (2) (p^{\ast})^{1} (1 - p^{\ast})^{1} = 2 (p^{\ast})(1 - p^{\ast})$
- $p(i = 0) = (1) (p^{\ast})^{0} (1 - p^{\ast})^{2} = (1 - p^{\ast})^{2}$

<br>

The binomial pmf is available from the `Scipy` module. We can import it with the following:

In [22]:
from scipy.stats import binom

The PMF is available as a method:

In [23]:
binom.pmf??

Signature: binom.pmf(k, *args, **kwds)
Source:   
    def pmf(self, k, *args, **kwds):
        """
        Probability mass function at k of the given RV.

        Parameters
        ----------
        k : array_like
            Quantiles.
        arg1, arg2, arg3,... : array_like
            The shape parameter(s) for the distribution (see docstring of the
            instance object for more information)
        loc : array_like, optional
            Location parameter (default=0).

        Returns
        -------
        pmf : array_like
            Probability mass function evaluated at k

        """
        args, loc, _ = self._parse_args(*args, **kwds)
        k, loc = map(asarray, (k, loc))
        args = tuple(map(asarray, args))
        k = asarray((k-loc))
        cond0 = self._argcheck(*args)
        cond1 = (k >= self.a) & (k <= self.b) & self._nonzero(k, *args)
        cond = cond0 & cond1
        output = zeros(shape(cond), 'd')
        place(output, (1-cond0) + np.isnan

In [24]:
binom.pmf(2, n, pstar)

0.19995651425667552

In [25]:
pstar * pstar

0.19995651425667552

In [26]:
binom.pmf(1, n, pstar)

0.49441692012233124

In [27]:
2 * pstar * (1 - pstar)

0.4944169201223312

In [28]:
binom.pmf(0, n, pstar)

0.3056265656209933

In [29]:
(1 - pstar) ** 2

0.3056265656209932

You guessed it: YEET!! Now let's add this to our code. Again, starting at the bottom and moving up.

In [30]:
probs = np.zeros((nodes, ))
for i in range(nodes):
    probs[i] = binom.pmf(i, n, pstar)
probs

array([0.30562657, 0.49441692, 0.19995651])

In [31]:
spotT

array([ 70.87417468, 108.32870677, 165.57665416])

We've now got an array with terminal prices and an array with terminal probabilities. Let's get option payoffs and take the dot product.

In [32]:
callT = callPayoff(spotT, K)
callT

array([ 0.        , 13.32870677, 70.57665416])

In [34]:
np.sum(callT * probs)

20.702199902328218

Now we just need a present value.

In [35]:
callPrc = np.exp(-r * expiry) * np.dot(callT, probs)
callPrc

19.11053913600299

It's different because we're now using two periods instead of just a single period. 

<br>

Let's put this in a function.

In [ ]:
def binomialPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    for i in range(nodes):
        prob = binom.pmf(i, n, pstar)
        spotT = S * (u ** i) * (d ** (n - i))
        po = payoff(spotT, K) 
        price += po * prob
        if verbose:
            print(f"({spotT:0.4f}, {po:0.4f}, {prob:0.4f})")
        
    price *= np.exp(-r * T)
    
    return price

In [38]:
binomialPricer(S, K, r, v, q, expiry, n, callPayoff)

(70.8742, 0.0000, 0.3056)
(108.3287, 13.3287, 0.4944)
(165.5767, 70.5767, 0.2000)


19.11053913600299

In [40]:
n = 2
binomialPricer(S, K, r, v, q, expiry, n, putPayoff)

(70.8742, 24.1258, 0.3056)
(108.3287, 0.0000, 0.4944)
(165.5767, 0.0000, 0.2000)


6.806592042733392

We can now set $n$ arbitrarily large! But let's start with $n = 3$

In [41]:
n = 3

In [42]:
binomialPricer(S, K, r, v, q, expiry, n, callPayoff)

(64.4284, 0.0000, 0.1603)
(91.1007, 0.0000, 0.4044)
(128.8147, 33.8147, 0.3400)
(182.1418, 87.1418, 0.0953)


18.282552207370557

In [43]:
binomialPricer(S, K, r, v, q, expiry, n, putPayoff)

(64.4284, 30.5716, 0.1603)
(91.1007, 3.8993, 0.4044)
(128.8147, 0.0000, 0.3400)
(182.1418, 0.0000, 0.0953)


5.9786051141009695

Now really big!

In [45]:
n = 500
callPrc = binomialPricer(S, K, r, v, q, expiry, n, callPayoff, verbose=False)
print(f"The Call Price is: {callPrc:0.4f}")

The Call Price is: 18.3856


In [46]:
putPrc = binomialPricer(S, K, r, v, q, expiry, n, putPayoff, verbose=False)
print(f"The Put Price is: {putPrc:0.4f}")

The Put Price is: 6.0816


In [49]:
## Compare to Examples 12.1 and 12.2 in McDonald Chapter 12

S = 41.0
K = 40.0
r = 0.08
v = 0.3
q = 0.0
T = 0.25

In [51]:
for n in range(1, 21):
    callPrc = binomialPricer(S, K, r, v, q, T, n, callPayoff, verbose=False)
    print(f"The Call Premium is: {callPrc:0.4f}")

The Call Premium is: 3.8982
The Call Premium is: 3.4676
The Call Premium is: 3.4976
The Call Premium is: 3.4848
The Call Premium is: 3.4153
The Call Premium is: 3.4755
The Call Premium is: 3.3802
The Call Premium is: 3.4644
The Call Premium is: 3.3608
The Call Premium is: 3.4544
The Call Premium is: 3.3485
The Call Premium is: 3.4458
The Call Premium is: 3.3659
The Call Premium is: 3.4384
The Call Premium is: 3.3806
The Call Premium is: 3.4320
The Call Premium is: 3.3907
The Call Premium is: 3.4263
The Call Premium is: 3.3977
The Call Premium is: 3.4213


In [55]:
n = 10000
callPrc = binomialPricer(S, K, r, v, q, T, n, callPayoff, verbose=False)
callPrc

3.3990613316590794

In [57]:
putPrc = binomialPricer(S, K, r, v, q, T, n, putPayoff, verbose=False)
putPrc

1.6070082639258576

In [58]:
def binomialPricer(S, K, r, v, q, T, n, payoff):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    spotT = np.empty((nodes, ))
    po = np.empty((nodes, ))
    prob = np.empty((nodes,))
    
    for i in range(nodes):
        prob[i] = binom.pmf(i, n, pstar)
        spotT[i] = S * (u ** i) * (d ** (n - i))
        po[i] = payoff(spotT, K) 
          
    price = np.exp(-r * T) * np.dot(po, prob)
    
    return price

In [59]:
np.exp(-0.08)

0.9231163463866358